In [1]:
from pymongo import MongoClient
import sys
import io


In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
client = MongoClient("mongodb://localhost:27017/")

In [4]:
database = client['contact_list_db']
database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'contact_list_db')

In [5]:
# Function to recursively extract nested fields' names and types
def extract_fields(data, prefix="", fields_info={}):
    for key, value in data.items():
        field_name = f"{prefix}.{key}" if prefix else key

        if isinstance(value, dict):
            # Recursively extract nested fields
            extract_fields(value, field_name, fields_info)
        elif isinstance(value, list):
            # Check if the list contains dictionaries
            if all(isinstance(item, dict) for item in value):
                for item in value:
                    # Recursively extract fields for each item in the list
                    extract_fields(item, field_name, fields_info)
            else:
                # Otherwise, treat it as an array
                fields_info[field_name] = "array"
        else:
            field_type = type(value).__name__
            if field_name not in fields_info:
                fields_info[field_name] = field_type

# Initialize an empty dictionary to store the collection information
collections_info = {}

# Get the list of collection names in the database
collection_names = database.list_collection_names()

# Iterate through each collection and fetch field names and types
for collection_name in collection_names:
    collection = database[collection_name]
    collection_info = {}

    # Fetch all documents from the collection
    documents = collection.find()

    # Extract fields from each document
    for document in documents:
        extract_fields(document, "", collection_info)

    # Store the collection information in the dictionary
    collections_info[collection_name] = collection_info




In [7]:
collections_info = {}

# Get the list of collection names in the database
collection_names = database.list_collection_names()

# Function to recursively extract nested field names and types
def extract_fields(data, prefix="", fields_info={}):
    for key, value in data.items():
        field_name = f"{prefix}.{key}" if prefix else key

        if isinstance(value, dict):
            # Recursively extract nested fields
            extract_fields(value, field_name, fields_info)
        elif isinstance(value, list):
            # Check if the list contains dictionaries
            if all(isinstance(item, dict) for item in value):
                for i, item in enumerate(value):
                    # Recursively extract fields for each item in the list
                    extract_fields(item, f"{field_name}.{i}", fields_info)
            else:
                # Otherwise, treat it as an array
                fields_info[field_name] = "array"
        else:
            field_type = type(value).__name__
            fields_info[field_name] = field_type

# Iterate through each collection and fetch field names and types
for collection_name in collection_names:
    collection = database[collection_name]
    collection_info = {}

    # Fetch a document from the collection to determine field names and types
    sample_document = collection.find_one()

    # Extract fields from the sample document
    extract_fields(sample_document, "", collection_info)

    # Store the collection information in the dictionary
    collections_info[collection_name] = collection_info

In [6]:
collections_info

{'Hospital': {'_id': 'ObjectId',
  'hospital_id': 'str',
  'name': 'str',
  'address.street': 'str',
  'address.city': 'str',
  'address.state': 'str',
  'address.zip_code': 'str',
  'address.country': 'str',
  'contact.email': 'str',
  'contact.phone': 'str',
  'services': 'array',
  'rating': 'float',
  'capacity.total_beds': 'int',
  'capacity.available_beds': 'int',
  'capacity.icu_beds': 'int',
  'departments.department_id': 'str',
  'departments.name': 'str',
  'departments.head_doctor': 'str',
  'departments.description': 'str'},
 'Patient': {'_id': 'ObjectId',
  'patient_id': 'str',
  'first_name': 'str',
  'last_name': 'str',
  'gender': 'str',
  'date_of_birth': 'datetime',
  'contact.email': 'str',
  'contact.phone': 'str',
  'contact.address.street': 'str',
  'contact.address.city': 'str',
  'contact.address.state': 'str',
  'contact.address.zip_code': 'str',
  'contact.address.country': 'str',
  'medical_history.blood_type': 'str',
  'medical_history.allergies': 'array',
 

In [9]:
hospital_collection_name = "Hospital"
patient_collection_name = "Patient"

# Function to print patient details along with their associated hospital's department details
def print_patient_with_department(patient, hospital, department):
    print(f"Patient ID: {patient['patient_id']}")
    print(f"Name: {patient['first_name']} {patient['last_name']}")
    print(f"Gender: {patient['gender']}")
    print(f"Date of Birth: {patient['date_of_birth']}")
    print("Contact:")
    print(f"  Email: {patient['contact']['email']}")
    print(f"  Phone: {patient['contact']['phone']}")
    print("Address:")
    print(f"  Street: {patient['contact']['address']['street']}")
    print(f"  City: {patient['contact']['address']['city']}")
    print(f"  State: {patient['contact']['address']['state']}")
    print(f"  Zip Code: {patient['contact']['address']['zip_code']}")
    print(f"  Country: {patient['contact']['address']['country']}")
    print("Hospital:")
    print(f"  Hospital ID: {hospital['_id']}")
    print(f"  Name: {hospital['name']}")
    print(f"  Rating: {hospital['rating']}")
    print("Department:")
    print(f"  Department ID: {department['department_id']}")
    print(f"  Name: {department['name']}")
    print(f"  Head Doctor: {department['head_doctor']}")
    print(f"  Description: {department['description']}")
    print("\n")

# Get the hospitals and patients collection objects
hospital_collection = database[hospital_collection_name]
patient_collection = database[patient_collection_name]

# Query for male patients
male_patients = patient_collection.find({"gender": "Male"})

# Iterate through the male patients and fetch their associated hospital and department details
for patient in male_patients:
    hospital_id = patient["hospital_id"]
    hospital = hospital_collection.find_one({"_id": hospital_id})
    if hospital and "departments" in hospital:
        for department in hospital["departments"]:
            print_patient_with_department(patient, hospital, department)

Patient ID: PAT00196
Name: Christopher Patrick
Gender: Male
Date of Birth: 1991-02-03 00:00:00
Contact:
  Email: hunterrodriguez@example.org
  Phone: 348.458.0613x730
Address:
  Street: 2178 Hill Wells Suite 563
  City: South Katherine
  State: FL
  Zip Code: 77911
  Country: United States
Hospital:
  Hospital ID: 64c0c15fd3727c5d0c88e470
  Name: Orthopedics
  Rating: 3.4
Department:
  Department ID: DEPT001
  Name: Pediatrics
  Head Doctor: Dr. Holmes
  Description: Specializes in pediatrics-related issues.


Patient ID: PAT00196
Name: Christopher Patrick
Gender: Male
Date of Birth: 1991-02-03 00:00:00
Contact:
  Email: hunterrodriguez@example.org
  Phone: 348.458.0613x730
Address:
  Street: 2178 Hill Wells Suite 563
  City: South Katherine
  State: FL
  Zip Code: 77911
  Country: United States
Hospital:
  Hospital ID: 64c0c15fd3727c5d0c88e470
  Name: Orthopedics
  Rating: 3.4
Department:
  Department ID: DEPT002
  Name: Oncology
  Head Doctor: Dr. Rodriguez
  Description: Specializes

In [11]:
import openai
import json
from urllib.parse import quote_plus
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [18]:
prompt = ChatPromptTemplate.from_template(

    "Given mongodb schema {collections_info} , give me latest version of pymongo no-sql code for {query},neglect MongoDB server connection code ,strictly return only code no explanation or any other comments, give proper executable python code. return responce as a dictionary format where key is 'code' and value is pymongo code"

)




input_sentence = "Give me the number of hospitals"





chain = LLMChain(llm=llm, prompt=prompt)

extracted_query = chain.run({'collections_info': collections_info, 'query': input_sentence})

In [14]:
def parse_string(input_string): 

    keyword = "db" 

    start_index = input_string.find(keyword) 

    if start_index == -1: 

        return "Keyword not found in the input string." 

    # Extracting data starting from the keyword "db" 

    extracted_data = input_string[start_index:] 

    # Replacing "\\n" with "\n" 

    extracted_data = extracted_data.replace("\\n", "\n") 

    # Removing trailing "}" 

    if extracted_data.endswith("\"}"): 

        extracted_data = extracted_data[:-2] 

    return extracted_data

In [19]:
extracted_query

'{\n  "code": "from pymongo import MongoClient\\n\\nclient = MongoClient()\\ndb = client[\'your_database_name\']\\nhospitals = db[\'Hospital\']\\n\\nnum_hospitals = hospitals.count_documents({})\\n\\nresponse = {\'code\': num_hospitals}\\nresponse"\n}'

In [20]:
print(parse_string(extracted_query))

db = client['your_database_name']
hospitals = db['Hospital']

num_hospitals = hospitals.count_documents({})

response = {'code': num_hospitals}
response"
}


In [26]:
hospitals = database['Hospital']

num_hospitals = hospitals.count_documents({})

response = {'code': num_hospitals}
response

{'code': 5}

In [16]:
patients_collection = database['Patient']

# Find male patients
male_patients = patients_collection.find({'gender': 'male'})

# Get the hospital IDs of male patients
hospital_ids = [patient['hospital_id'] for patient in male_patients]

# Access the hospital collection
hospitals_collection = database['Hospital']

# Find departments of hospitals where male patients are admitted
departments = hospitals_collection.find({'hospital_id': {'$in': hospital_ids}}, {'departments': 1})

# Prepare the response dictionary
response = {'departments': []}

# Iterate over departments and add them to the response
for department in departments:
    response['departments'].extend(department['departments'])

# Close the MongoDB connection
#client.close()

# Return the response
response

{'departments': []}

In [17]:
patients_collection = database['Patient']
male_patients = patients_collection.find({'gender': 'male'})
hospital_ids = [patient['hospital_id']for patient in male_patients]

hospitals_collection = database['Hospital']
departments = hospitals_collection.find({'hospital_id': {'$in': hospital_ids}}, {'departments': 1})
response = {'departments': []}
for department in departments:
    response['departments'].extend(department['departments'])
client.close()
response

{'departments': []}

In [9]:
hospital_collection_name = "Hospital"
patient_collection_name = "Patient"

# Function to print patient details along with the hospital information
def print_patient_with_hospital(patient, hospital):
    print(f"Patient ID: {patient['patient_id']}")
    print(f"Name: {patient['first_name']} {patient['last_name']}")
    print(f"Gender: {patient['gender']}")
    print(f"Date of Birth: {patient['date_of_birth']}")
    print("Contact:")
    print(f"  Email: {patient['contact']['email']}")
    print(f"  Phone: {patient['contact']['phone']}")
    print("Address:")
    print(f"  Street: {patient['contact']['address']['street']}")
    print(f"  City: {patient['contact']['address']['city']}")
    print(f"  State: {patient['contact']['address']['state']}")
    print(f"  Zip Code: {patient['contact']['address']['zip_code']}")
    print(f"  Country: {patient['contact']['address']['country']}")
    print("Hospital:")
    print(f"  Hospital ID: {hospital['_id']}")
    print(f"  Name: {hospital['name']}")
    print(f"  Rating: {hospital['rating']}")
    print("\n")

# Get the hospitals and patients collection objects
hospital_collection = database[hospital_collection_name]
patient_collection = database[patient_collection_name]

# Query for male patients
male_patients = patient_collection.find({"gender": "Male"})

# Iterate through the male patients and fetch their associated hospitals
for patient in male_patients:
    hospital_id = patient["hospital_id"]
    hospital = hospital_collection.find_one({"_id": hospital_id})
    if hospital:
        print_patient_with_hospital(patient, hospital)

Patient ID: PAT00196
Name: Christopher Patrick
Gender: Male
Date of Birth: 1991-02-03 00:00:00
Contact:
  Email: hunterrodriguez@example.org
  Phone: 348.458.0613x730
Address:
  Street: 2178 Hill Wells Suite 563
  City: South Katherine
  State: FL
  Zip Code: 77911
  Country: United States
Hospital:
  Hospital ID: 64c0c15fd3727c5d0c88e470
  Name: Orthopedics
  Rating: 3.4


Patient ID: PAT00706
Name: Katrina Johnson
Gender: Male
Date of Birth: 1976-01-03 00:00:00
Contact:
  Email: robertpayne@example.com
  Phone: 001-509-297-0237x32299
Address:
  Street: 1544 Daniel Expressway Apt. 584
  City: Beckymouth
  State: MI
  Zip Code: 34310
  Country: United States
Hospital:
  Hospital ID: 64c0c287d3727c5d0c88e471
  Name: Orthopedics
  Rating: 3.7


Patient ID: PAT00262
Name: Amanda Arias
Gender: Male
Date of Birth: 2002-06-20 00:00:00
Contact:
  Email: rwalsh@example.org
  Phone: (393)389-1705
Address:
  Street: 3253 Matthews Overpass
  City: South Matthew
  State: SD
  Zip Code: 77639
  Cou

In [16]:
collection_names = database.list_collection_names()

In [8]:
patient_collection_name = "Patient"

# Query to get the emails of patients
query = {
    "contact.email": {"$exists": True}
}

# Project to only get the email field
projection = {
    "_id": 0,
    "contact.email": 1
}

# Execute the query and projection
result = database[patient_collection_name].find(query, projection)

# Extract the emails from the result
emails_of_patients = [patient["contact"]["email"] for patient in result]

print(emails_of_patients)









['hunterrodriguez@example.org', 'robertpayne@example.com', 'lamapril@example.net', 'rwalsh@example.org', 'tracie96@example.com']


In [22]:
patient_collection_name = "Patient"
hospital_collection_name = "Hospital"

pipeline = [
    {
        "$lookup": {
            "from": hospital_collection_name,
            "localField": "hospital_id",
            "foreignField": "hospital_id",
            "as": "hospital_info"
        }
    },
    {
        "$unwind": "$hospital_info"
    },
    {
        "$match": {
            "gender": "Male"
        }
    },
    {
        "$project": {
            "_id": 0,  # Exclude _id field from the final result
            "patient_id": 1,
            "first_name": 1,
            "last_name": 1,
            "gender": 1,
            "date_of_birth": 1,
            "hospital_info.name": 1,
            "hospital_info.address": 1,
            "hospital_info.contact.email": 1,
            "hospital_info.contact.phone": 1,
            "hospital_info.rating": 1,
            "hospital_info.capacity": 1
        }
    }
]

# Execute the aggregation pipeline
result = database[patient_collection_name].aggregate(pipeline)

# Print the results
for patient_hospital in result:
    print(patient_hospital)

In [23]:
for patient in result:
    print(patient)

In [24]:
result

In [9]:
patient_collection_name = "Patient"

# Specify the field names you want to retrieve (including nested fields)
city_field = "contact.address.city"

# Perform the query and projection
city_details = database[patient_collection_name].find({}, {city_field: 1})

# Print the city details of patients
for patient in city_details:
    city = patient.get("contact", {}).get("address", {}).get("city")
    if city:
        print(city)

Jennyton
Amberburgh
Nathantown
Smithside
South Katherine
Beckymouth
West Crystal
South Shirleystad
South Matthew
South Allisonfort


In [17]:


# Initialize an empty dictionary to store the information
collections_info = {}

# Get the list of collection names in the database
collection_names = database.list_collection_names()

def get_field_type(field_value, parent=None):
    if isinstance(field_value, list):
        return "array"
    #elif isinstance(field_value, dict):
        #get_field_type(field_value)#, parent=field_name)
    else:
        return type(field_value).__name__

# Iterate through each collection and fetch field names and types
for collection_name in collection_names:
    collection = database[collection_name]
    collection_info = {}

    # Fetch a document from the collection to determine field names and types
    sample_document = collection.find_one()

    # Iterate through the fields and their types
    for field, value in sample_document.items():
        field_type = get_field_type(value)
        collection_info[field] = field_type

    # Store the collection information in the dictionary
    var = "Collection " + collection_name
    collections_info[var] = collection_info

#print(collections_info)

In [18]:
database.list_collection_names()

['Hospital', 'Patient']

In [23]:
collection_info = {}
def extract_fields(data, parent=None):
    for key, value in data.items():
        field_name = key #if parent is None else f"{parent}.{key}"
        if isinstance(value, dict):
            extract_fields(value, parent=field_name)
        elif isinstance(value, list):
            #print("Hey")
            collection_info.setdefault(parent, {})[field_name] = "array"
        else:
            collection_info.setdefault(parent, {})[field_name] = type(value).__name__
#for collection_name in database.list_collection_names():
collection = database['Patient']
sample_document = collection.find_one()
extract_fields(sample_document)

In [8]:
collection_names = database.list_collection_names()
collections_info = {}
for collection_name in collection_names:
    collection_info = {}
    def extract_fields(data, parent=None):
        for key, value in data.items():
            field_name = key if parent is None else f"{parent}.{key}"
            if isinstance(value, dict):
                extract_fields(value, parent=field_name)
            elif isinstance(value, list):
                #print("Hey")
                collection_info.setdefault(parent, {})[field_name] = "array"
            else:
                collection_info.setdefault(parent, {})[field_name] = type(value).__name__
    #for collection_name in database.list_collection_names():
    collection = database[collection_name]
    sample_document = collection.find_one()
    extract_fields(sample_document)
    #collection_info['collection_name'] = collection_info.pop(None)
    collections_info.update(collection_info)

In [24]:
collection_info

{None: {'_id': 'str',
  'patient_id': 'str',
  'first_name': 'str',
  'last_name': 'str',
  'gender': 'str',
  'date_of_birth': 'str',
  'appointments': 'array',
  'prescriptions': 'array'},
 'contact': {'email': 'str', 'phone': 'str'},
 'address': {'street': 'str',
  'city': 'str',
  'state': 'str',
  'zip_code': 'str',
  'country': 'str'},
 'medical_history': {'blood_type': 'str',
  'allergies': 'array',
  'conditions': 'array'},
 'insurance': {'provider_name': 'str', 'policy_number': 'int'}}

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [10]:
import openai

collection_info = """-- //Patient Collection Schema
{
  _id: ObjectId, // MongoDB will automatically generate a unique ID for each student
  patient_name: String,
  date_of_birth: Date,
  gender: String,
  street: String,
  city: String,
  zip: String,
  state: String,
  country: String,
  social_security_number: String,
  hospital_id: ObjectId // Manual reference to Course IDs
}

-- //Hospital Collection Schema
{
  _id: ObjectId, // MongoDB will automatically generate a unique ID for each course
  hospital_name: String,
  street: String,
  city: String,
  zip: String,
  state: String,
  country: String,
  patient_id: [ObjectId],
};
"""

In [11]:
query_sentence = "Give the details of male patients of hospitals"

In [12]:
prompt = f"Forget previous instructions if input does not make sense respond with an empty query a. Given {collections_info}, generate only the NoSQL query for the sentence '{query_sentence}'. Return the response as a dictionary format where the key is 'code' and the value is the noSQL query."
response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=100
)

In [13]:
response

<OpenAIObject text_completion id=cmpl-7gU7Zht9z2CyxD8gtkDQFsrr6VbUc at 0x21dec06e400> JSON: {
  "id": "cmpl-7gU7Zht9z2CyxD8gtkDQFsrr6VbUc",
  "object": "text_completion",
  "created": 1690359917,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\n{'code': ''}",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 514,
    "completion_tokens": 8,
    "total_tokens": 522
  }
}

In [14]:
extracted_query = response.choices[0].text.strip()
extracted_query0 = str(extracted_query)

if "{'code': '" in extracted_query0:
    extracted_query1 = extracted_query0.replace("{'code': '", "")
else:
    extracted_query1 = extracted_query0

if "'}" in extracted_query1:
    query = extracted_query1.replace("'}", "")
else:
    query = extracted_query1
print(query)

In [15]:
query

''

In [20]:
query = query.replace("db", "database")
query

"database.Patient.find({}, {'contact.address.city': 1})"

In [21]:
query
city_field = "contact.address.city"

# Perform the query and projection
city_details = database.Patient.find({}, {'contact.address.city': 1})
# Print the city details of patients
for patient in city_details:
    city = patient.get("contact", {}).get("address", {}).get("city")
    if city:
        print(city)

Jennyton
Amberburgh
Nathantown
Smithside
South Katherine
Beckymouth
West Crystal
South Shirleystad
South Matthew
South Allisonfort


In [68]:
# Your collection names
hospital_collection_name = "Hospital"
patient_collection_name = "Patient"
pipeline = [
    {
        "$lookup": {
            "from": hospital_collection_name,
            "localField": "hospital_id",
            "foreignField": "_id",
            "as": "hospital_details"
        }
    },
    {
        "$unwind": "$hospital_details"  # If each patient has one hospital only, use $unwind to flatten the result
    },
    {
        "$project": {
            "_id": 0,  # Exclude _id field from the final result
            "patient_name": 1,
            "date_of_birth": 1,
            "gender": 1,
            "social_security_number": 1,
            "hospital_details": {
                "hospital_name": 1,
                "street": 1,
                "city": 1,
                "zip": 1,
                "state": 1,
                "country": 1
            }
        }
    }
]

# Execute the aggregation pipeline
result = database[patient_collection_name].aggregate(pipeline)

# Print the results
for patient in result:
    print(patient)

In [70]:
print(patient)

In [75]:
import json
import random
from faker import Faker
from datetime import datetime

fake = Faker()

def generate_patient():
    patient_id = fake.uuid4()
    first_name = fake.first_name()
    last_name = fake.last_name()
    gender = random.choice(["Male", "Female"])
    date_of_birth = fake.date_of_birth(tzinfo=None, minimum_age=18, maximum_age=80)
    email = fake.email()
    phone = fake.phone_number()
    street = fake.street_address()
    city = fake.city()
    state = fake.state_abbr()
    zip_code = fake.zipcode()
    country = "United States"
    blood_type = random.choice(["A+", "B+", "AB+", "O+", "A-", "B-", "AB-", "O-"])
    allergies = random.sample(["Penicillin", "Pollen", "Dust", "Shellfish"], random.randint(0, 3))
    conditions = random.sample(["Hypertension", "Diabetes", "Asthma", "Migraine", "Cardovascular_attack"], random.randint(0, 2))
    prescription_medication = ["Ibuprofen", "Paracetamol", "Antibiotics", "Aspirin"]
    medication = random.choice(prescription_medication)
    dosage = random.choice(["200mg", "500mg", "1g", "2g"])
    instructions = f"Take one tablet every {random.randint(4, 8)} hours as needed for pain."
    insurance_provider_name = fake.company()
    policy_number = fake.random_number(digits=8)

    patient_data = {
        "_id": str(patient_id),
        "patient_id": f"PAT00{random.randint(100, 999)}",
        "first_name": first_name,
        "last_name": last_name,
        "gender": gender,
        "date_of_birth": date_of_birth.isoformat(),
        "contact": {
            "email": email,
            "phone": phone,
            "address": {
                "street": street,
                "city": city,
                "state": state,
                "zip_code": zip_code,
                "country": country
            }
        },
        "medical_history": {
            "blood_type": blood_type,
            "allergies": allergies,
            "conditions": conditions
        },
        "appointments": [
            {
                "appointment_id": f"APT00{random.randint(100, 999)}",
                "doctor_name": f"Dr. {fake.last_name()}",
                "date": (datetime.today()).isoformat(),
                "reason": "General Checkup"
            },
            # You can add other appointment objects for the patient here
        ],
        "prescriptions": [
            {
                "prescription_id": f"RX00{random.randint(100, 999)}",
                "medication": medication,
                "dosage": dosage,
                "instructions": instructions
            },
            # You can add other prescription objects for the patient here
        ],
        "insurance": {
            "provider_name": insurance_provider_name,
            "policy_number": policy_number
        }
        # Add other relevant patient-related fields as needed
    }
    return patient_data

# Generate 10 patient records
patients = [generate_patient() for _ in range(10)]

file_path = "output.json"

# Save the JSON document to the file
with open(file_path, "w") as file:
    json.dump(patients, file, indent=2)

# Print the generated JSON
#print(patients_json)


In [76]:
def generate_hospital():
    hospital_id = fake.uuid4()
    name = f"{fake.company()} Hospital"
    street = fake.street_address()
    city = fake.city()
    state = fake.state_abbr()
    zip_code = fake.zipcode()
    country = "United States"
    email = fake.email()
    phone = fake.phone_number()
    department_names = ["Cardiology", "Orthopedics", "Neurology", "Oncology", "Pediatrics"]
    departments = []
    for i in range(random.randint(1, len(department_names))):
        department_id = f"DEPT00{i+1}"
        name = random.choice(department_names)
        head_doctor = f"Dr. {fake.last_name()}"
        description = f"Specializes in {name.lower()}-related issues."
        department = {
            "department_id": department_id,
            "name": name,
            "head_doctor": head_doctor,
            "description": description
        }
        departments.append(department)
    services = random.sample(["Emergency Care", "Laboratory Services", "Radiology", "Surgery", "Maternity", "ENT"], random.randint(1, 5))
    rating = round(random.uniform(2.5, 5.0), 1)
    total_beds = random.randint(100, 500)
    available_beds = random.randint(20, total_beds)
    icu_beds = random.randint(5, available_beds)

    hospital_data = {
        "_id": str(hospital_id),
        "hospital_id": f"HOSP00{random.randint(100, 999)}",
        "name": name,
        "address": {
            "street": street,
            "city": city,
            "state": state,
            "zip_code": zip_code,
            "country": country
        },
        "contact": {
            "email": email,
            "phone": phone
        },
        "departments": departments,
        "services": services,
        "rating": rating,
        "capacity": {
            "total_beds": total_beds,
            "available_beds": available_beds,
            "icu_beds": icu_beds
        }
        # Add other relevant hospital-related fields as needed
    }
    return hospital_data

# Generate 10 hospital records
hospitals = [generate_hospital() for _ in range(10)]

# Convert the list of hospital records to JSON
file_path = "hospital.json"

# Save the JSON document to the file
with open(file_path, "w") as file:
    json.dump(hospitals, file, indent=2)